# Обратная задача ЭЭГ

Обратная задача ЭЭГ заключается в восстановлении распределения источников (тока и заряда) внутри области (например, головы) на основе измерений электрического потенциала на границе.

В данном ноутбуке мы используем нейросеть для моделирования распределения заряда и тока, а также обновляем функцию потерь и пайплайн для решения этой задачи.

In [1]:
# Импорт необходимых библиотек
using NeuralPDE, Lux, ModelingToolkit, Optimization, OptimizationOptimJL, LineSearches,
      OptimizationOptimisers, LuxCUDA, Random, ComponentArrays
using ModelingToolkit: Interval, infimum, supremum
using Distributions, Plots, CUDA

using Random
using TensorBoardLogger

In [2]:
const gpud = gpu_device()
const cpud = cpu_device()

(::CPUDevice) (generic function with 4 methods)

In [3]:
# Определение прямой задачи
@parameters x, y, z, t
@variables φ(..), Ax(..),Ay(..),Az(..), ρ(..), jx(..), jy(..), jz(..)
A = [Ax, Ay, Az]
j = [jx, jy, jz]
Dxx = Differential(x)^2
Dyy = Differential(y)^2
Dzz = Differential(z)^2

# Определение физических постоянных
const c = 2.99792458e10 # Скорость света в вакууме (см/с)
const ε₀ = 1.0 # Диэлектрическая постоянная вакуума в СГС (размерность отсутствует)
const ε = 1.0  # Диэлектрическая проницаемость (отн.)
const μ₀ = 1.0 # Магнитная постоянная вакуума в СГС (размерность отсутствует)
const μ = 1.0  # Магнитная проницаемость (отн.)


# Определение оператора Лапласа как функции
function laplacian(F, params)
    return sum((Differential(param)^2)(F) for param in params)
end

# Определение оператора Даламбера как функции
function dalembert_operator(F, params, ε, μ, c)
    Δ = laplacian(F, params)
    return Δ  - (ε * μ / c^2) * (Differential(t)^2)(F)
end


dalembert_operator (generic function with 1 method)

In [4]:
dalembert_operator(φ(x,y,z,t), [x, y, z], ε, μ, c)

Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t)))

In [5]:
# Уравнение
eqs = [

    [dalembert_operator(φ(x, y, z, t), [x, y, z], ε, μ, c) ~ -4 * pi * ρ(x, y, z, t) / ε];
    [dalembert_operator(A[i](x, y, z, t), [x, y, z], ε, μ, c) ~ -μ * 4 * pi / c* j[i](x, y, z, t) for i in 1:3];
    (Differential(x)(Ax(x, y, z, t)) + Differential(y)(Ay(x, y, z, t)) + Differential(z)(Az(x, y, z, t)) + (ε * μ / c) * Differential(t)(φ(x, y, z, t))) ~ 0.0
]

5-element Vector{Equation}:
 Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t))) ~ -12.566370614359172ρ(x, y, z, t)
 Differential(x)(Differential(x)(Ax(x, y, z, t))) + Differential(z)(Differential(z)(Ax(x, y, z, t))) + Differential(y)(Differential(y)(Ax(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ax(x, y, z, t))) ~ -4.191690043903363e-10jx(x, y, z, t)
 Differential(y)(Differential(y)(Ay(x, y, z, t))) + Differential(z)(Differential(z)(Ay(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ay(x, y, z, t))) + Differential(x)(Differential(x)(Ay(x, y, z, t))) ~ -4.191690043903363e-10jy(x, y, z, t)
 Differential(z)(Differential(z)(Az(x, y, z, t))) + Differential(y)(Differential(y)(Az(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Az(x, y, z, t))) + Differential

In [6]:
# Границы области
const x_min, x_max = -10.0, 10.0
const y_min, y_max = -10.0, 10.0
const z_min, z_max = -10.0, 10.0
const t_min, t_max = 0.0, 1.0
# Область
domains = [x ∈ Interval(x_min, x_max),
           y ∈ Interval(y_min, y_max),
           z ∈ Interval(z_min, z_max), 
           t ∈ Interval(t_min, t_max)]
# Начальные условия

function analytic_sol_func(t, x, y, z)
    r = sqrt((x)^2 + (y)^2 + (z)^2)
    (t + 1)^2 / r
end

# Генерация случайных точек в пределах домена
num_points = 100
measured_points = []
for _ in 1:num_points
    x_p = rand(x_min/2:x_max/2)
    y_p = rand(y_min/2:y_max/2)
    z_p = rand(z_min/2:z_max/2)
    t_p = rand(t_min/2:t_max/2)
    # Добавление случайной точки в массив measured_points
    phi_p = analytic_sol_func(t_p, x_p, y_p, z_p)
    push!(measured_points, [x_p, y_p, z_p, t_p, phi_p])
end
measured_points = measured_points |> gpud
bcs = [
    [φ(x_min, y, z, t) ~ 0.0, φ(x_max, y, z, t) ~ 0.0,
    φ(x, y_min, z, t) ~ 0.0, φ(x, y_max, z, t) ~ 0.0,
    φ(x, y, z_min, t) ~ 0.0, φ(x, y, z_max, t) ~ 0.0];
    [A[i](x_min, y, z, t)  ~ 0.0 for i in 1:3]
]


9-element Vector{Equation}:
 φ(-10.0, y, z, t) ~ 0.0
 φ(10.0, y, z, t) ~ 0.0
 φ(x, -10.0, z, t) ~ 0.0
 φ(x, 10.0, z, t) ~ 0.0
 φ(x, y, -10.0, t) ~ 0.0
 φ(x, y, 10.0, t) ~ 0.0
 Ax(-10.0, y, z, t) ~ 0.0
 Ay(-10.0, y, z, t) ~ 0.0
 Az(-10.0, y, z, t) ~ 0.0

In [7]:
measured_points

100-element Vector{CuArray{Float32, 1, CUDA.DeviceMemory}}:
 Float32[3.0, 3.0, -1.0, 0.0, 0.22941573]
 Float32[2.0, 3.0, -3.0, 0.0, 0.21320072]
 Float32[2.0, 4.0, 5.0, 0.0, 0.1490712]
 Float32[-3.0, 0.0, -5.0, 0.0, 0.17149858]
 Float32[0.0, 3.0, 5.0, 0.0, 0.17149858]
 Float32[-4.0, -1.0, -1.0, 0.0, 0.23570226]
 Float32[3.0, -2.0, 2.0, 0.0, 0.24253562]
 Float32[4.0, -5.0, 4.0, 0.0, 0.13245323]
 Float32[4.0, -1.0, 0.0, 0.0, 0.24253562]
 Float32[-2.0, -2.0, 2.0, 0.0, 0.28867513]
 Float32[-2.0, 4.0, -3.0, 0.0, 0.18569534]
 Float32[-4.0, 3.0, -5.0, 0.0, 0.14142136]
 Float32[2.0, 0.0, -2.0, 0.0, 0.35355338]
 ⋮
 Float32[-1.0, 3.0, 1.0, 0.0, 0.30151135]
 Float32[1.0, -3.0, 5.0, 0.0, 0.16903085]
 Float32[-5.0, 3.0, 0.0, 0.0, 0.17149858]
 Float32[-2.0, -5.0, 1.0, 0.0, 0.18257418]
 Float32[-2.0, 0.0, 3.0, 0.0, 0.2773501]
 Float32[2.0, -2.0, -4.0, 0.0, 0.20412415]
 Float32[4.0, 2.0, 0.0, 0.0, 0.2236068]
 Float32[0.0, 5.0, -5.0, 0.0, 0.14142136]
 Float32[4.0, -1.0, 2.0, 0.0, 0.2182179]
 Float32[-2.

In [8]:
function additional_loss_weightened(lambda)
    
    function additional_loss(phi_pred_fun, θ, p_)
        CUDA.allowscalar() do
            # phi is first output of phi_pred_fun
            result = sum(abs2(phi_pred_fun([x, y, z, t]|>cpud, θ|>cpud)[1] - phi|>cpud) for (x, y, z, t, phi) in measured_points) / length(measured_points)|>cpud
            result = result * lambda
            return result
        end
    end
    
    return additional_loss
end

additional_loss_weightened (generic function with 1 method)

In [9]:

# Нейросеть
# Определение новой нейросети для моделирования распределения заряда и тока
input_ = 4  # x, y, z
n = 32      # число нейронов в скрытых слоях
lambda = 10 # вес дополнительной потерь
# Функция для разделения выхода сети на переменные
"""
function split_outputs(out)
    φ_pred = out[1]
    A_pred = out[2:4]
    ρ_pred = out[5]
    j_pred = out[6:8]
    return φ_pred, A_pred, ρ_pred, j_pred
end

chain = [Chain(
    Dense(input_, n, σ),
    Dense(n, n, σ),
    Dense(n, 1)
) for _ in 1:8] 

# Определение системы
ps = [Lux.setup(Random.default_rng(), chain[i])[1] |> ComponentArray |> gpud .|> Float64 for i in 1:8]
"""
chain = Chain(
    Dense(input_, n, σ),
    Dense(n, n, σ),
    Dense(n, n, σ),
    Dense(n, 8)
)

# Определение системы
ps = Lux.setup(Random.default_rng(), chain)[1] |> ComponentArray |> gpud .|> Float32

strategy = QuasiRandomTraining(4096)
discretization = PhysicsInformedNN(chain, strategy; init_params = ps, additional_loss = additional_loss_weightened(lambda), 
log_options = LogOptions(; log_frequency = 50))

PhysicsInformedNN{Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, QuasiRandomTraining{QuasiMonteCarlo.LatinHypercubeSample}, ComponentVector{Float32, CuArray{Float32, 1, CUDA.DeviceMemory}, Tuple{Axis{(layer_1 = ViewAxis(1:160, Axis(weight = ViewAxis(1:128, ShapedAxis((32, 4))), bias = ViewAxis(129:160, Shaped1DAxis((32,))))), layer_2 = ViewAxis(161:1216, Axis(weight = ViewAxis(1:1024, ShapedAxis((32, 32))), bias = ViewAxis(1025:1056, Shaped1DAxis((32,))))), layer_3 = ViewAxis(1217:2272, Axis(weight = ViewAxis(1:1024, ShapedAxis((32, 32))), bias = ViewAxis(1025:1056, Shaped1DAxis((32,))))), layer_4 = ViewAxis(2273:2536, Axis(weight = ViewAxis(1:256, ShapedAxis((8, 32))), bias = ViewAxis(257:264, Shaped1DAxis((8,)))

In [10]:
additional_loss_weightened(lambda)(discretization.phi, ps, 0)

5.0265117f0

In [11]:
allvars = [φ(x, y, z, t); [A_(x, y, z, t) for A_ in A]; ρ(x, y, z, t); [j_(x, y, z, t) for j_ in j]]

8-element Vector{Num}:
  φ(x, y, z, t)
 Ax(x, y, z, t)
 Ay(x, y, z, t)
 Az(x, y, z, t)
  ρ(x, y, z, t)
 jx(x, y, z, t)
 jy(x, y, z, t)
 jz(x, y, z, t)

In [12]:
@named pde_system = PDESystem(eqs, bcs, domains, [x, y, z, t], allvars)

PDESystem
Equations: Equation[Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t))) ~ -12.566370614359172ρ(x, y, z, t), Differential(x)(Differential(x)(Ax(x, y, z, t))) + Differential(z)(Differential(z)(Ax(x, y, z, t))) + Differential(y)(Differential(y)(Ax(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ax(x, y, z, t))) ~ -4.191690043903363e-10jx(x, y, z, t), Differential(y)(Differential(y)(Ay(x, y, z, t))) + Differential(z)(Differential(z)(Ay(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ay(x, y, z, t))) + Differential(x)(Differential(x)(Ay(x, y, z, t))) ~ -4.191690043903363e-10jy(x, y, z, t), Differential(z)(Differential(z)(Az(x, y, z, t))) + Differential(y)(Differential(y)(Az(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Az(x, y, z, t))) + Differentia

In [13]:
prob = discretize(pde_system, discretization)
sym_prob = symbolic_discretize(pde_system, discretization)


NeuralPDE.PINNRepresentation(Equation[Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t))) ~ -12.566370614359172ρ(x, y, z, t), Differential(x)(Differential(x)(Ax(x, y, z, t))) + Differential(z)(Differential(z)(Ax(x, y, z, t))) + Differential(y)(Differential(y)(Ax(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ax(x, y, z, t))) ~ -4.191690043903363e-10jx(x, y, z, t), Differential(y)(Differential(y)(Ay(x, y, z, t))) + Differential(z)(Differential(z)(Ay(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ay(x, y, z, t))) + Differential(x)(Differential(x)(Ay(x, y, z, t))) ~ -4.191690043903363e-10jy(x, y, z, t), Differential(z)(Differential(z)(Az(x, y, z, t))) + Differential(y)(Differential(y)(Az(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Az(x, y, z, t))) + Dif

In [21]:
#phi = discretization.phi
pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions
add_functon = sym_prob.loss_functions.additional_loss_function



# Создаем логгер для TensorBoard
logger = TBLogger("../../logs/inverse_npde_exp")

function create_callback()
    iter = 0  # Локальный счетчик итераций

    return function (p, l)
        
        iter += 1  # Увеличиваем номер итерации
        
        # Логируем общую потерю
        log_value(logger, "Loss/Total", l; step=iter)
        
        # Логируем потери по PDE
        pde_losses = map(l_ -> l_(p.u), pde_inner_loss_functions)
        for (i, pde_loss) in enumerate(pde_losses)
            log_value(logger, "Loss/PDE_$i", pde_loss; step=iter)
        end
        
        # Логируем потери по граничным условиям
        bcs_losses = map(l_ -> l_(p.u), bcs_inner_loss_functions)
        for (i, bc_loss) in enumerate(bcs_losses)
            log_value(logger, "Loss/BC_$i", bc_loss; step=iter)
        end
        
        # Выводим информацию в консоль
        println("Iteration: $iter, Total Loss: $l")
        println("PDE Losses: ", pde_losses)
        println("BC Losses: ", bcs_losses)
        
        return false
    end
end

create_callback (generic function with 1 method)

In [22]:
callback = create_callback()

#22 (generic function with 1 method)

In [24]:
# Оптимизация
opt = OptimizationOptimisers.Adam(0.001)

Adam(0.001, (0.9, 0.999), 1.0e-8)

In [25]:

# Решение
res = solve(prob, opt; maxiters = 100, callback)
phi = discretization.phi

Iteration: 2, Total Loss: 32.620009136009585
PDE Losses: [26.08448274869011, 5.112024676015553e-6, 5.196069098985501e-6, 5.1098342774461885e-6, 3.2560078804921945e-5]
BC Losses: [0.16840384601949607, 0.16209392460857575, 0.1538349991126832, 0.17756767392898776, 0.15247310921255391, 0.17889902632147903, 0.16846200083314958, 0.16843226472519712, 0.16845072231507316]
[26.08448274869011, 5.112024676015553e-6, 5.196069098985501e-6, 5.1098342774461885e-6, 3.2560078804921945e-5]
BC Losses: [0.16840384601949607, 0.16209392460857575, 0.1538349991126832, 0.17756767392898776, 0.15247310921255391, 0.17889902632147903, 0.16846200083314958, 0.16843226472519712, 0.16845072231507316]
Iteration: 3, Total Loss: 29.43963123314975
PDE Losses: [23.55060941452313, 5.213018030316391e-6, 5.113438606671548e-6, 5.088027216532092e-6, 3.215359027682133e-5]
BC Losses: [0.1524209109470089, 0.14599582726957355, 0.13811862023069682, 0.16102730200749105, 0.1373201236199592, 0.16192458734113563, 0.152387941132107, 0.15

NeuralPDE.Phi{StatefulLuxLayer{Static.True, Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, Nothing, @NamedTuple{layer_1::@NamedTuple{}, layer_2::@NamedTuple{}, layer_3::@NamedTuple{}, layer_4::@NamedTuple{}}}}(StatefulLuxLayer{Static.True, Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, Nothing, @NamedTuple{layer_1::@NamedTuple{}, layer_2::@NamedTuple{}, layer_3::@NamedTuple{}, layer_4::@NamedTuple{}}}(Chain{@NamedTuple{layer_1::Dense{typeof(σ

In [ ]:
res.u

ComponentVector{Float32, CuArray{Float32, 1, CUDA.DeviceMemory}, Tuple{Axis{(layer_1 = ViewAxis(1:160, Axis(weight = ViewAxis(1:128, ShapedAxis((32, 4))), bias = ViewAxis(129:160, Shaped1DAxis((32,))))), layer_2 = ViewAxis(161:1216, Axis(weight = ViewAxis(1:1024, ShapedAxis((32, 32))), bias = ViewAxis(1025:1056, Shaped1DAxis((32,))))), layer_3 = ViewAxis(1217:2272, Axis(weight = ViewAxis(1:1024, ShapedAxis((32, 32))), bias = ViewAxis(1025:1056, Shaped1DAxis((32,))))), layer_4 = ViewAxis(2273:2536, Axis(weight = ViewAxis(1:256, ShapedAxis((8, 32))), bias = ViewAxis(257:264, Shaped1DAxis((8,))))))}}}(layer_1 = (weight = Float32[0.64270455 -0.69251525 -0.21865846 -0.6302313; 0.09410353 0.16299486 0.14028741 -0.17518775; … ; 0.6903682 -0.73030674 0.43923947 -0.6788331; 0.7310284 -0.6661543 0.18700586 0.3957257], bias = Float32[-0.05248393, -0.40303862, 0.29772493, 0.0049014185, 0.06712001, 0.38455158, 0.29028347, 0.46409968, -0.3953544, 0.27702054  …  0.038982, -0.45193848, -0.036569875, 0

In [ ]:
using Plots

phi = discretization.phi
xs, ys, zs, ts = [infimum(d.domain):0.01:supremum(d.domain) for d in domains]

minimizers_ = res.u|> cpud
z_selected = 0.0
t_selected = 0.0
clip = 10
u_real = [clamp(analytic_sol_func(0, xs, ys, z_selected), -clip, clip) for xs in xs for ys in ys]
u_predict = [(phi([x, y, z_selected, t_selected], minimizers_))[1] for x in xs for y in ys ]
diff_u = [abs.(u_real .- u_predict)]

ps = []

p1 = plot(xs, ys, u_real, linetype = :contourf, title = "phi, analytic")
p2 = plot(xs, ys, u_predict, linetype = :contourf, title = "phi, predict")
p3 = plot(xs, ys, diff_u, linetype = :contourf, title = "phi, error")
push!(ps, plot(p1, p2, p3))


# Сохранение графиков
savefig(ps[1], "../../figures/plot_phi.png")


LoadError: SystemError: opening file "/home/sasha/inverse-npde/experiments/base/figures/plot_phi.png": Нет такого файла или каталога

"/home/sasha/inverse-npde/figures/plot_phi.png"